<a href="https://colab.research.google.com/github/eshhacb/Election-Sentiment-Classifier/blob/main/USA_Election_Syntiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px

In [17]:
trump_reviews = pd.read_csv("Trumpall2.csv")
biden_reviews = pd.read_csv("Bidenall2.csv")

In [18]:
trump_reviews.head(3)

,user,text
0,manny_rosen,@sanofi please tell us how many shares the Cr...
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P..."
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...


In [19]:
biden_reviews.head(3)

,user,text
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...
2,OldlaceA,@JoeBiden So did Lying Barr


# Sentiment Analysis
Now, I will get started with sentiment analysis. I will do it by using the Textblob package in Python. Here I will use this package to perform simple text classification in either positive or negative on the basis of sentiment analysis:

1 Polarity refers to the sentiment or emotion expressed in the text, ranging from negative to positive.


2 Subjectivity,suggests that the text contains some subjects or opinions, measures the extent to which the text is subjective or opinionated rather than objective

In [20]:
textblob1 = TextBlob(trump_reviews['text'][10])
textblob1.sentiment
textblob2 = TextBlob(biden_reviews['text'][500])
textblob2.sentiment

Sentiment(polarity=0.6, subjectivity=0.9)

In [21]:
def find_pol(review):
    return TextBlob(review).sentiment.polarity

In [22]:
trump_reviews["Sentiment Polarity"] = trump_reviews["text"].apply(find_pol)
biden_reviews["Sentiment Polarity"] = biden_reviews["text"].apply(find_pol)

In [25]:
trump_reviews.head(3)

,user,text,Sentiment Polarity
0,manny_rosen,@sanofi please tell us how many shares the Cr...,0.05
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P...",0.00
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...,-0.50


In [26]:
biden_reviews.head(3)

,user,text,Sentiment Polarity
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...,0.0
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...,0.0
2,OldlaceA,@JoeBiden So did Lying Barr,0.0


# Sentiment Polarity on Both the candidates:
Now I will add a new attribute in both the datasets by the name of “Expression Label”:                                        
1 polarity > 0 :: positive                                                               
2 polarity < 0 :: negative                                                     
3 Polarity == 0 :: neutral

In [28]:
trump_reviews["Expression Label"] = np.where(trump_reviews["Sentiment Polarity"]>0, "positive", "negative")
trump_reviews.loc[trump_reviews["Sentiment Polarity"]==0,"Expression Label"]="Neutral"
print(trump_reviews.tail())

biden_reviews["Expression Label"] = np.where(biden_reviews["Sentiment Polarity"]>0, "positive", "negative")
biden_reviews.loc[biden_reviews["Sentiment Polarity"]==0,"Expression Label"]="Neutral"
print(biden_reviews.tail())

                 user                                               text  \
2783          4diva63  @realDonaldTrump For the 1/100 time, absentee ...   
2784         hidge826  @realDonaldTrump If you’re so scared of losing...   
2785     SpencerRossy  @realDonaldTrump I rarely get involved with fo...   
2786  ScoobyMcpherson  @realDonaldTrump This is the moment when Trump...   
2787          bjklinz     @realDonaldTrump I’m sorry, Donald. No. #POTUS   

      Sentiment Polarity Expression Label  
2783               0.000          Neutral  
2784               0.000          Neutral  
2785               0.225         positive  
2786               0.000          Neutral  
2787              -0.500         negative  
             user                                               text  \
2535    meryn1977  @JoeBiden You'll just try to calm those waters...   
2536  BSNelson114  @JoeBiden 96 days 96 dias #VoteJoeBiden2020  #...   
2537     KenCapel  @JoeBiden YOU THINK YOU CAN DO THAT??? YOU C

# Drop all the tweets with neutral polarity

In [29]:
reviews1 = trump_reviews[trump_reviews['Sentiment Polarity'] == 0.0000]
print(reviews1.shape)

cond1=trump_reviews['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
trump_reviews.drop(trump_reviews[cond1].index, inplace = True)
print(trump_reviews.shape)

(1464, 4)
(1324, 4)


In [30]:
reviews2 = biden_reviews[biden_reviews['Sentiment Polarity'] == 0.0000]
print(reviews2.shape)

cond2=biden_reviews['Sentiment Polarity'].isin(reviews1['Sentiment Polarity'])
biden_reviews.drop(biden_reviews[cond2].index, inplace = True)
print(biden_reviews.shape)

(1509, 4)
(1031, 4)


# balance both the datasets

In [31]:
# Donald Trump
np.random.seed(10)
remove_n =324
drop_indices = np.random.choice(trump_reviews.index, remove_n, replace=False)
df_subset_trump = trump_reviews.drop(drop_indices)
print(df_subset_trump.shape)
# Joe Biden
np.random.seed(10)
remove_n =31
drop_indices = np.random.choice(biden_reviews.index, remove_n, replace=False)
df_subset_biden = biden_reviews.drop(drop_indices)
print(df_subset_biden.shape)

(1000, 4)
(1000, 4)


# predict the US Elections, by analyzing the number of positive and negative sentiments in both the accounts:

In [33]:
count_1 = df_subset_trump.groupby('Expression Label').count()
print(count_1)

negative_per1 = (count_1['Sentiment Polarity'].iloc[0]/1000)*10
positive_per1 = (count_1['Sentiment Polarity'].iloc[1]/1000)*100

count_2 = df_subset_biden.groupby('Expression Label').count()
print(count_2)

negative_per2 = (count_2['Sentiment Polarity'].iloc[0]/1000)*100
positive_per2 = (count_2['Sentiment Polarity'].iloc[1]/1000)*100

                  user  text  Sentiment Polarity
Expression Label                                
negative           449   449                 449
positive           551   551                 551
                  user  text  Sentiment Polarity
Expression Label                                
negative           393   393                 393
positive           607   607                 607


In [34]:
Politicians = ['Joe Biden', 'Donald Trump']
lis_pos = [positive_per1, positive_per2]
lis_neg = [negative_per1, negative_per2]

fig = go.Figure(data=[
    go.Bar(name='Positive', x=Politicians, y=lis_pos),
    go.Bar(name='Negative', x=Politicians, y=lis_neg)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()